In [1]:
import os
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn import metrics
from scipy.stats import entropy, kurtosis
import xgboost as xgb
import seaborn as sns
from xgboost import plot_importance
import warnings
import matplotlib.pyplot as plt
import pandas as pd
from math import *
import numpy as np
import tensorflow as tf
import time
import gc

pd.set_option('display.max_columns', None)
from IPython.display import display

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
pathf = os.path.join("..", "data", "particles")
model_path = os.path.join(pathf, "model")
log_path = os.path.join(pathf, "model")
trainpd = pd.read_csv(os.path.join(pathf, "train.csv"))
print(trainpd.head(1))
trainshape = trainpd.shape
print(trainshape)
eventpd = pd.read_csv(os.path.join(pathf, "event.csv"))
print(eventpd.head(1))
print(eventpd.shape)
testpd = pd.read_csv(os.path.join(pathf, "test.csv"))
testshape = testpd.shape
print(testpd.head(1))
print(testpd.shape)

data = pd.concat([trainpd, testpd], ignore_index=True)
data = pd.merge(data, eventpd, on='event_id', how='left')

       x      y  z        t   terror        q  flag  event_id  hit_id
0 -142.5 -147.5  0  767.879  2.02966  1.05052     0         7       1
(9473201, 9)
   event_id  nhit  nhitreal  energymc  thetamc    phimc   xcmc    ycmc
0         7   426        70   48348.9  63.1686  11.0982 -40.83  114.03
(13315, 8)
       x      y  z        t  terror        q  event_id  hit_id
0 -142.5 -127.5  0  848.061  1.9984  1.15067         9       1
(4086511, 8)


In [3]:
# (k(q,mc)*(t0+L))^2 + dis^2 -dis*cos(phi)*sin(thmc)*(t0+L) = (t+L)^2
# t0 方程 
# a = k(q,mc)^2
# b = 2*L*k(q,mc)^2 -dis*cos(phi)*sin(thmc)
# c = L^2 * k(q,mc)^2 + dis^2 - dis*cos(phi)*sin(thmc)*L - (t+L)^2 
# t0 = (-b +- (b^2 - 4*a*c)^(1/2))/2*a
data = pd.concat([trainpd, testpd], ignore_index=True)
data = pd.merge(data, eventpd, on='event_id', how='left')

data['fx'] = data['x'] - data['xcmc']
data['fy'] = data['y'] - data['ycmc']
data['phimc'] = data['phimc'] * np.pi / 180.
data['fphi'] = np.arctan2(data['fy'], data['fx']) - data['phimc']
data['fdis'] = np.sqrt(data['fx'] ** 2 + data['fy'] ** 2)
data['thetamc'] = data['thetamc'] * np.pi / 180.

data['fsinthmc'] = np.sin(data['thetamc'])
data['fsinthmc_v'] = 1.0/data['fsinthmc']
data['fcosphi'] = np.cos(data['fphi'])
data['fcosphi_v'] = 1.0/data['fcosphi']

data['fcosthmc'] = np.cos(data['thetamc'])
data['fcosthmc_v'] = 1.0/data['fcosthmc']
data['fsinphi'] = np.sin(data['fphi'])
data['fsinphi_v'] = 1.0/data['fsinphi']

data['ftanphi'] = np.tan(data['fphi'])
data['ftanphi_v'] = 1.0/data['ftanphi']
data['ftanthmc'] = np.tan(data['thetamc'])
data['ftanthmc_v'] = 1.0/data['ftanthmc']


# data['ft2'] = data['t'] ** 2
# data['fdis2'] = data['fdis'] ** 2

data['fttrue'] = data['t'] / data['terror']
data['fttrue_v'] = 1.0 / data['fttrue']
data['fttrue2'] = data['fttrue'] ** 2
data['fttrue2_v'] = 1.0 / data['fttrue2'] 
data['nhitratio'] = data['nhit'] / data['nhitreal']
data['nhitratio_v'] = data['nhitratio']
data['energymc_v'] = 1.0 / data['energymc']
data['fenergymc2'] = data['energymc'] ** 2
data['fenergymc2_v'] = 1.0 / data['fenergymc2'] 
data['q_v'] = 1.0 / data['q']
data['q2'] = data['q']
data['q2_v'] = 1.0 / data['q2']

del data['fx']
del data['fy']
del data['x']
del data['y']
del data['z']
del data['xcmc']
del data['ycmc']
del data['fphi']
del data['phimc']
del data['fsinphi']
del data['nhitreal']
gc.collect()

33

In [4]:
info_new = pd.DataFrame()
info_new["event_id"] = data.groupby(["event_id"])["event_id"].mean()
info_new["fdis_mean"] = data.groupby(["event_id"])["fdis"].mean()
info_new["fdis_std"] = data.groupby(["event_id"])["fdis"].std()
info_new["fdis_stdmean"] = info_new["fdis_std"] / info_new["fdis_mean"]
info_new["ft_min"] = data.groupby(["event_id"])["t"].min()
info_new["ft_max"] = data.groupby(["event_id"])["t"].max()
info_new["t_mean"] = data.groupby(["event_id"])["t"].mean()
info_new["ft_std"] = data.groupby(["event_id"])["t"].std()
info_new["ft_stdmean"] = info_new["ft_std"] / info_new["t_mean"]
info_new["ft_mean"] = (info_new['t_mean']-info_new['ft_min']) / (info_new['ft_max']-info_new['ft_min'])
info_new.reset_index(drop=True, inplace=True)
data = pd.merge(data, info_new, on='event_id', how='left')

data['ft_rel'] = data['t'] / data['ft_std']
data['ft2_rel'] = data['ft_rel'] ** 2
data['ft_rel_v'] = 1.0 / data['ft_rel']
data['ft2_rel_v'] = 1.0 / data['ft2_rel'] 

# (k(q,mc)*(t0+L))^2 + dis^2 -dis*cos(phi)*sin(thmc)*(t0+L) = (t+L)^2
data = data.sort_values(by=['event_id', 'ft_rel']).reset_index(drop=True)
for i in [1, 5, 7, 11]:
    data[f'ft_{i}diff'] = data.groupby('event_id')['ft_rel'].diff(periods=i).fillna(0)
del data['t']
gc.collect()

0

In [5]:
print(trainshape[0])
print(data.shape)
testpd = data[data.flag.isna()].reset_index()
trainpd = data[data.flag.notna()].reset_index()
trainpd['flag'] = trainpd['flag'].astype('int')
# trainpd = data[:trainshape[0]].reset_index()
# testpd = data[trainshape[0]:].reset_index()
del data
gc.collect()

9473201
(13559712, 49)


0

In [6]:
print(trainpd.columns)
feature = [x for x in trainpd.columns if x not in ['flag', 'index', 'hit_id', 'event_id']]
labels = trainpd['flag']
del trainpd['flag']
del testpd['flag']

Index(['index', 'terror', 'q', 'flag', 'event_id', 'hit_id', 'nhit',
       'energymc', 'thetamc', 'fdis', 'fsinthmc', 'fsinthmc_v', 'fcosphi',
       'fcosphi_v', 'fcosthmc', 'fcosthmc_v', 'fsinphi_v', 'ftanphi',
       'ftanphi_v', 'ftanthmc', 'ftanthmc_v', 'fttrue', 'fttrue_v', 'fttrue2',
       'fttrue2_v', 'nhitratio', 'nhitratio_v', 'energymc_v', 'fenergymc2',
       'fenergymc2_v', 'q_v', 'q2', 'q2_v', 'fdis_mean', 'fdis_std',
       'fdis_stdmean', 'ft_min', 'ft_max', 't_mean', 'ft_std', 'ft_stdmean',
       'ft_mean', 'ft_rel', 'ft2_rel', 'ft_rel_v', 'ft2_rel_v', 'ft_1diff',
       'ft_5diff', 'ft_7diff', 'ft_11diff'],
      dtype='object')


In [7]:
print(trainpd.head())
npx = trainpd.values  # returns a numpy array
npx[np.isinf(npx)] = 0 # 清洗nan
min_max_scaler = preprocessing.MinMaxScaler()
trainnormalpd = pd.DataFrame([[0]*len(trainpd.columns)])
trainnormalpd= pd.DataFrame(min_max_scaler.fit_transform(npx))
trainnormalpd.columns = trainpd.columns
print(trainnormalpd.head())

   index   terror         q  event_id  hit_id  nhit  energymc  thetamc  \
0      0  2.02966  1.450270         7     219   426   48348.9   1.1025   
1      1  1.68831  3.100560         7     138   426   48348.9   1.1025   
2      2  2.02966  1.098740         7     198   426   48348.9   1.1025   
3      3  2.02966  0.799937         7      87   426   48348.9   1.1025   
4      4  2.02966  0.748987         7      98   426   48348.9   1.1025   

         fdis  fsinthmc  fsinthmc_v   fcosphi  fcosphi_v  fcosthmc  \
0   34.491010  0.892339    1.120651  0.423789   2.359667  0.451367   
1  105.107230  0.892339    1.120651 -0.650026  -1.538400  0.451367   
2  105.408395  0.892339    1.120651  0.078329  12.766613  0.451367   
3  196.706456  0.892339    1.120651 -0.150763  -6.632925  0.451367   
4  126.053282  0.892339    1.120651 -0.814480  -1.227777  0.451367   

   fcosthmc_v  fsinphi_v    ftanphi  ftanphi_v  ftanthmc  ftanthmc_v  \
0    2.215494   1.104044   2.137295   0.467881  1.976971    0.

In [8]:
def batch_iter_list(data_list, batch_size, num_epochs, shuffle=True):
    data_size = len(data_list[0])
    num_batches_per_epoch = data_size // batch_size  # 每个epoch中包含的batch数量
    for epoch in range(num_epochs):
        # 每个epoch是否进行shuflle
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data_list = [data[shuffle_indices] for data in data_list]
        else:
            shuffled_data_list = data_list

        for batch_num in range(num_batches_per_epoch + 1):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield [shuffled_data[start_index:end_index] for shuffled_data in shuffled_data_list]

class AbstractModeltensor(object):
    def __init__(self, config=None):
        self.config = config

    # You need to override this method.
    def buildModel(self):
        raise NotImplementedError("You need to implement your own model.")


class NeurousNet(AbstractModeltensor):
    def __init__(self, xlenth, config=None):
        super(NeurousNet, self).__init__(config)
        self.graph = tf.Graph()  # 为每个类(实例)单独创建一个graph
        self.modeldic = {
            "cnn_dense_less": self._cnn_dense_less_model,
            "mul4_model": self._mul4_model,
            "nomul_model": self._nomul_model,
        }
        self.ydim = 1
        self.keep_prob_ph = config["dropout"]
        self.input_dim = xlenth
        self.out_dim = 1
        with self.graph.as_default():
            with tf.name_scope('Inputs'):
                self.input_p = tf.placeholder(tf.float32, [None, self.input_dim])
                self.learn_rate_p = tf.placeholder(dtype=tf.float32, shape=[], name="lr")
                self.lr_decay = tf.placeholder(dtype=tf.float32, shape=[])
            with tf.name_scope('Outputs'):
                self.target_y = tf.placeholder(dtype=tf.float32, shape=[None, self.out_dim])

    def buildModel(self):
        tf.reset_default_graph()
        with self.graph.as_default():
            # 不同选择加载
            self.modeldic[self.config["modelname"]]()
            # 打印打包
            self.merged = tf.summary.merge_all()
            # 损失目标
            tvars = tf.trainable_variables()  # 返回需要训练的variable
            grads, _ = tf.clip_by_global_norm(tf.gradients(self.train_list, tvars), 1e-3)
            grads_and_vars = tuple(zip(grads, tvars))
            self.train_op = tf.train.AdamOptimizer(self.learn_rate_p).apply_gradients(grads_and_vars)
            #             self.train_op = []
            #             for i2 in self.train_list:
            #                 self.train_op.append(tf.train.AdamOptimizer(self.learn_rate_p).minimize(i2))
            # 同一保存加载
            self.saver = tf.train.Saver(tf.global_variables())
            # [print(n.name) for n in tf.get_default_graph().as_graph_def().node]
            # return self.saver

    def _mul4_model(self):
        with self.graph.as_default():
            # 部分1，预测值
            base0 = tf.layers.dense(inputs=self.input_p, units=self.input_dim*4, activation=tf.nn.elu,
                                     name="base0")
            base1 = tf.layers.dense(inputs=self.input_p, units=self.input_dim*8, activation=tf.nn.elu,
                                     name="base1")
            base2 = tf.layers.dense(inputs=self.input_p, units=self.input_dim*16, activation=tf.nn.elu,
                                     name="base2")
            base3 = tf.layers.dense(inputs=self.input_p, units=self.input_dim*32, activation=tf.nn.elu,
                                     name="base3")
            mult0 = tf.layers.dense(inputs=self.input_p, units=self.input_dim*4, activation=tf.nn.elu,
                                     name="mult0")
            mult_o1 = tf.nn.elu(mult0 * base0, name='mult_o1') # 4 in
            mult_e1 = tf.layers.dense(inputs=mult_o1, units=self.input_dim*8, activation=tf.nn.elu,
                                     name="mult_e1")
            mult_o2 = tf.nn.elu(mult_e1 * base1, name='mult_o2') # 8 in
            mult_e2 = tf.layers.dense(inputs=mult_o2, units=self.input_dim*16, activation=tf.nn.elu,
                                     name="mult_e2")
            mult_o3 = tf.nn.elu(mult_e2 * base2, name='mult_o3') # 16 in
            mult_e3 = tf.layers.dense(inputs=mult_o3, units=self.input_dim*32, activation=tf.nn.elu,
                                     name="mult_e3")            
            mult_o4 = tf.nn.elu(mult_e3 * base3, name='mult_o4') # 32 in
            
            concat1 = tf.concat([self.input_p, mult_o1, mult_o2, mult_o3, mult_o4], 1, name='concat1')
            denseo1 = tf.nn.dropout(concat1, keep_prob=self.keep_prob_ph)
            denseo2 = tf.layers.dense(inputs=denseo1, units=self.input_dim * 16, activation=tf.nn.elu, 
                                      name="denseo2")
            denseo2 = tf.nn.dropout(denseo2, keep_prob=self.keep_prob_ph)
            denseo3 = tf.layers.dense(inputs=denseo2, units=self.input_dim, activation=tf.nn.elu,
                                      name="denseo3")
            denseo4 = tf.layers.dense(inputs=denseo3, units=self.input_dim // 8, activation=tf.nn.elu,
                                      name="denseo4")
            y_res_t = tf.layers.dense(inputs=denseo4, units=self.out_dim, activation=None)
            y_res_v = tf.nn.sigmoid(y_res_t, name="y_res_v")
            tf.summary.histogram('y_res_v', y_res_v)  # 记录标量的变化
            # 损失返回值
            y_los = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_res_t, labels=self.target_y, name="y_los")
            y_loss_t = tf.reduce_mean(y_los, name="y_loss_t")
            y_loss_v = tf.add(y_loss_t, 0, name="y_loss_v")

            one = tf.ones_like(y_res_t)
            zero = tf.zeros_like(y_res_t)
            label_bool = tf.where(y_res_t < 0.5, x=zero, y=one)
            self.auc_value, self.auc_op = tf.metrics.auc(self.target_y, label_bool, num_thresholds=4000)
            # 猜错的获取 实际盈利值的负数
            self.train_list = [y_loss_t]
            self.valid_list = [y_loss_v]
            self.pred_list = [y_res_v]
            # 打印信息
            tf.summary.scalar('y_loss_t', y_loss_t)  # 记录标量的变化
            tf.summary.scalar('y_loss_v', y_loss_v)  # 记录标量的变化
            tf.summary.histogram('concat1', concat1)  # 记录标量的变化
            tf.summary.histogram('denseo4', denseo4)  # 记录标量的变化

            tf.summary.scalar('lr', self.learn_rate_p)  # 记录标量的变化
            return None

    def _cnn_dense_less_model(self):
        with self.graph.as_default():
            # 部分1，预测值
            unified = tf.nn.softmax(self.input_p)
            dense1 = tf.layers.dense(inputs=unified, units=self.input_dim, activation=tf.nn.softmax,
                                     name="layer_dense1")
            tf.summary.histogram('dense1', dense1)  # 记录标量的变化
            mult_layer1 = tf.nn.softmax(dense1 * self.input_p, name='mult_layer1')
            mult_layer2 = tf.nn.softmax(mult_layer1 * self.input_p, name='mult_layer2')
            concat1 = tf.concat([self.input_p, dense1, mult_layer1, mult_layer2], 1, name='concat1')
            tf.summary.histogram('concat1', concat1)  # 记录标量的变化
            denseo1 = tf.nn.dropout(concat1, keep_prob=self.keep_prob_ph)
            denseo2 = tf.layers.dense(inputs=denseo1, units=self.input_dim, activation=tf.nn.elu, name="layer_dense2")
            denseo3 = tf.layers.dense(inputs=denseo2, units=self.input_dim // 4, activation=tf.nn.elu,
                                      name="layer_dense3")
            y_res_t = tf.layers.dense(inputs=denseo3, units=self.out_dim, activation=None)
            y_res_v = tf.nn.sigmoid(y_res_t, name="y_res_v")
            tf.summary.histogram('y_res_v', y_res_v)  # 记录标量的变化
            # 损失返回值
            y_los = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_res_t, labels=self.target_y, name="y_los")
            y_loss_t = tf.reduce_mean(y_los, name="y_loss_t")
            y_loss_v = tf.add(y_loss_t, 0, name="y_loss_v")

            one = tf.ones_like(y_res_t)
            zero = tf.zeros_like(y_res_t)
            label_bool = tf.where(y_res_t < 0.5, x=zero, y=one)
            self.auc_value, self.auc_op = tf.metrics.auc(self.target_y, label_bool, num_thresholds=4000)
            # 猜错的获取 实际盈利值的负数
            self.train_list = [y_loss_t]
            self.valid_list = [y_loss_v]
            self.pred_list = [y_res_v]
            # 打印信息
            tf.summary.scalar('y_loss_t', y_loss_t)  # 记录标量的变化
            tf.summary.scalar('y_loss_v', y_loss_v)  # 记录标量的变化
            tf.summary.histogram('mult_layer1', mult_layer1)  # 记录标量的变化
            tf.summary.histogram('mult_layer2', mult_layer2)  # 记录标量的变化

            tf.summary.scalar('lr', self.learn_rate_p)  # 记录标量的变化
            return None

    def _nomul_model(self):
        with self.graph.as_default():
            # 部分1，预测值
            dense1 = tf.layers.dense(inputs=self.input_p, units=self.input_dim, activation=tf.nn.softmax,
                                     name="layer_dense1")
            tf.summary.histogram('dense1', dense1)  # 记录标量的变化
            dense2 = tf.layers.dense(inputs=dense1, units=self.input_dim, activation=tf.nn.elu, name="layer_dense2")
            dense3 = tf.layers.dense(inputs=dense2, units=self.input_dim, activation=tf.nn.elu, name="layer_dense3")
            dense4 = tf.layers.dense(inputs=dense3, units=self.input_dim, activation=tf.nn.elu, name="layer_dense4")
            dense5 = tf.layers.dense(inputs=dense4, units=self.input_dim, activation=tf.nn.elu, name="layer_dense5")
            dense6 = tf.layers.dense(inputs=dense5, units=self.input_dim, activation=tf.nn.elu, name="layer_dense6")
            dense7 = tf.layers.dense(inputs=dense6, units=self.input_dim, activation=tf.nn.elu, name="layer_dense7")
            dense8 = tf.layers.dense(inputs=dense7, units=self.input_dim, activation=tf.nn.elu, name="layer_dense8")
            concat1 = tf.concat([self.input_p, dense1, dense2, dense3, dense4, dense5, dense6, dense7, dense8], 1,
                                name='concat1')
            tf.summary.histogram('concat1', concat1)  # 记录标量的变化
            denseo1 = tf.nn.dropout(concat1, keep_prob=self.keep_prob_ph)
            denseo2 = tf.layers.dense(inputs=denseo1, units=self.input_dim * 4, activation=tf.nn.elu,
                                      name="layer_denseo2")
            denseo3 = tf.layers.dense(inputs=denseo2, units=self.input_dim, activation=tf.nn.elu,
                                      name="layer_denseo3")
            denseo4 = tf.layers.dense(inputs=denseo3, units=self.input_dim // 4, activation=tf.nn.elu,
                                      name="layer_denseo4")
            y_res_t = tf.layers.dense(inputs=denseo4, units=self.out_dim, activation=None)
            y_res_v = tf.nn.sigmoid(y_res_t, name="y_res_v")
            tf.summary.histogram('y_res_v', y_res_v)  # 记录标量的变化
            # 损失返回值
            y_los = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_res_t, labels=self.target_y, name="y_los")
            y_loss_t = tf.reduce_mean(y_los, name="y_loss_t")
            y_loss_v = tf.add(y_loss_t, 0, name="y_loss_v")

            one = tf.ones_like(y_res_t)
            zero = tf.zeros_like(y_res_t)
            label_bool = tf.where(y_res_t < 0.5, x=zero, y=one)
            self.auc_value, self.auc_op = tf.metrics.auc(self.target_y, label_bool, num_thresholds=4000)
            # 猜错的获取 实际盈利值的负数
            self.train_list = [y_loss_t]
            self.valid_list = [y_loss_v]
            self.pred_list = [y_res_v]
            # 打印信息
            tf.summary.scalar('y_loss_t', y_loss_t)  # 记录标量的变化
            tf.summary.scalar('y_loss_v', y_loss_v)  # 记录标量的变化

            tf.summary.scalar('lr', self.learn_rate_p)  # 记录标量的变化
            return None

    def batch_train(self, trainpd, labels, batch_size=8, num_epochs=1, retrain=True):
        # 设置
        sess = tf.Session(graph=self.graph)
        with sess.as_default():
            with self.graph.as_default():
                if self.config["retrain"] == 1:
                    model_dir = os.path.join(model_path, "modelevery_%s" % self.config["tailname"])
                    latest_ckpt = tf.train.latest_checkpoint(model_dir)
                    if os.path.isfile("{}.index".format(latest_ckpt)):
                        self.saver.restore(sess, latest_ckpt)
                        sess.run(tf.local_variables_initializer())
                        print("retraining {}".format(latest_ckpt))
                    else:
                        sess.run(tf.global_variables_initializer())
                        sess.run(tf.local_variables_initializer())
                        print("no old model, training new----")
                writer = tf.summary.FileWriter(os.path.join(log_path, "logsevery_%s" % self.config["tailname"]),
                                               sess.graph)
                global_n = 0
                stop_n = 0
                startt = time.time()
                pre_t_base_loss = pre_t_much_loss = pre_v_much_loss = pre_v_base_loss = 100000

                n_splits = 5
                kf = KFold(n_splits=n_splits, shuffle=True, random_state=4389)
                for epoch in range(num_epochs):
                    for train_index, valid_index in kf.split(trainpd):
                        if self.config["learn_rate"]>1e-9:
                            self.config["learn_rate"] *= 0.7
                        inputs_t = np.array(trainpd[feature].iloc[train_index])
                        output_t = np.expand_dims(np.array(labels[train_index]),-1)
                        inputs_v = np.array(trainpd[feature].iloc[valid_index])
                        output_v = np.expand_dims(np.array(labels[valid_index]),-1)
                        dataiter = batch_iter_list([inputs_t,output_t], batch_size, num_epochs)
                        starte = time.time()
                        print("iter_trainnum", inputs_t.shape[0] // batch_size + 1)
                        redi = inputs_t.shape[0] % batch_size
                        lenth = inputs_t.shape[0] // batch_size
                        if 0 != redi:
                            lenth += 1
                        counter = 0
                        for batch_num in range(lenth):
                            # 获取数据
                            r_inputs_t,r_output_t = next(dataiter)
                            feed_dict_t = {
                                self.input_p: r_inputs_t,
                                self.target_y: r_output_t,
                                self.learn_rate_p: self.config["learn_rate"],
                                self.lr_decay: 1,
                            }
                            # 更新学习率
                            sess.run(self.train_op, feed_dict_t)
                            global_n += 1
                            losslist_t = sess.run(self.train_list, feed_dict_t)
                            sess.run(self.auc_op, feed_dict=feed_dict_t)
                            accu = sess.run(self.auc_value)
                            result = sess.run(self.merged, feed_dict_t)
                            if batch_num % 200 == 0:
                                writer.add_summary(result, global_n)
                                self.saver.save(sess,
                                                os.path.join(model_path, 'modelevery_%s' % self.config["tailname"],
                                                             self.config["modelfile"]), global_step=global_n)
                                print("epocht {}, batch_num {}, step {}, time: {} s, accu: {}, loss_yt: {}".format(
                                    epoch, batch_num, global_n, time.time() - starte, accu, *losslist_t))
                        # valid part
                        dataiterv = batch_iter_list([inputs_v,output_v], batch_size, num_epochs)
                        redi = inputs_v.shape[0] % batch_size
                        vnum_iter = inputs_v.shape[0] // batch_size
                        if 0 != redi:
                            vnum_iter += 1
                        counter = 0
                        print("iter_validnum", vnum_iter)
                        losslist_va = 0
                        accu_va = 0
                        dataiter = batch_iter_list([inputs_v,output_v], batch_size, num_epochs)
                        for batch_num in range(vnum_iter):
                            # 获取数据
                            r_inputs_v,r_output_v = next(dataiter)
                            feed_dict_v = {
                                self.input_p: r_inputs_v,
                                self.target_y: r_output_v,
                                self.learn_rate_p: self.config["learn_rate"],
                                self.lr_decay: 1,
                            }
                            losslist_v = sess.run(self.valid_list, feed_dict_v)
                            sess.run(self.auc_op, feed_dict=feed_dict_v)
                            accu = sess.run(self.auc_value)
                            losslist_va += losslist_v[0]
                            accu_va += accu
                        losslist_va /= vnum_iter
                        accu_va /= vnum_iter
                        result = sess.run(self.merged, feed_dict_v)
                        writer.add_summary(result, global_n)
                        if losslist_t[0] < pre_t_base_loss and losslist_va < pre_v_base_loss:
                            stop_n += 1
                            if stop_n > self.config["early_stop"]:
                                break
                            else:
                                self.saver.save(sess,
                                                os.path.join(model_path, 'modelevery_%s' % self.config["tailname"],
                                                             self.config["modelfile"]), global_step=global_n)
                        else:
                            stop_n = 0
                            self.saver.save(sess, os.path.join(model_path, 'modelevery_%s' % self.config["tailname"],
                                                               self.config["modelfile"]), global_step=global_n)
                        print("epochv {}, step {}, stop_n {}, time: {} s, accu_va: {}, loss_yv: {}".format(
                            epoch, global_n, stop_n, time.time() - starte, accu_va, losslist_va))
                        pre_t_base_loss = losslist_t[0]
                        pre_v_base_loss = losslist_va
                writer.close()
                print("total time: %s s" % (time.time() - startt))
        # 结束
        print("train finished!")
        return None

    def predict(self, inputs):
        model_dir = os.path.join(model_path, "modelevery_%s" % self.config["tailname"])
        print("loading model...")
        latest_ckpt = tf.train.latest_checkpoint(model_dir)

        sess = tf.Session(graph=self.graph)
        with sess.as_default():
            with self.graph.as_default():
                if os.path.isfile("{}.index".format(latest_ckpt)):
                    self.saver.restore(sess, latest_ckpt)
                else:
                    raise Exception("没有找到模型:{}".format(latest_ckpt))
                nplist = []
                oneiter = 2000
                redi = inputs.shape[0] % oneiter
                lenth = inputs.shape[0] // oneiter
                if 0 != redi:
                    lenth += 1
                counter = 0
                for num in range(lenth):
                    # 获取数据
                    startindex = num * oneiter
                    if num == lenth - 1 and redi != 0:
                        endindex = num * oneiter + redi
                    else:
                        endindex = (num + 1) * oneiter
                    tmppd = inputs.iloc[startindex:endindex][feature]
                    r_inputs_v = np.array(tmppd)
                    feed_dict = {
                        self.input_p: r_inputs_v,
                    }
                    teslis = sess.run(self.pred_list, feed_dict)
                    nplist.append(teslis)
                feed_dict = {
                    self.input_p: inputs,
                }
                teslist = np.concatenate(nplist, axis=1)
                return teslist


trainconfig = {
    "dropout": 0.5,
    "early_stop": 100,
#     "tailname": "nomul_modeltail",
#     "modelname": "nomul_model",
    "tailname": "mul4_modeltailv2",
    "modelname": "mul4_model",
#     "tailname": "mul_verse",
#     "modelname": "cnn_dense_less",
    "modelfile": "v2",
#     "learn_rate": 1e-3,
    "learn_rate": 5e-4,
#     "learn_rate": 1e-6,
    "retrain": 1
}
modelcrnn = NeurousNet(len(feature), config=trainconfig)
modelcrnn.buildModel()

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [9]:
# batch_size, num_epochs = 4096, 1000
# batch_size, num_epochs = 512, 1000
batch_size, num_epochs = 512, 1000
globalstep = modelcrnn.batch_train(trainnormalpd, labels, batch_size, num_epochs)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../data/particles/model/modelevery_mul4_modeltailv2/v2-7401
retraining ../data/particles/model/modelevery_mul4_modeltailv2/v2-7401
iter_trainnum 14802
epocht 0, batch_num 0, step 1, time: 13.664920806884766 s, accu: 0.9719613790512085, loss_yt: 0.08840502053499222
epocht 0, batch_num 200, step 201, time: 34.75502419471741 s, accu: 0.966947078704834, loss_yt: 0.07478544861078262
epocht 0, batch_num 400, step 401, time: 56.49523305892944 s, accu: 0.9673578143119812, loss_yt: 0.08808233588933945
epocht 0, batch_num 600, step 601, time: 78.4117681980133 s, accu: 0.9668822884559631, loss_yt: 0.06902296841144562
epocht 0, batch_num 800, step 801, time: 99.84157299995422 s, accu: 0.9669867157936096, loss_yt: 0.08117637783288956
Instructions for updating:
Use standard file APIs to delete files with this prefix.
epocht 0, batch_num 1000, step 1001, time: 121.359998703

epocht 0, batch_num 12400, step 12401, time: 1346.9222416877747 s, accu: 0.9674159288406372, loss_yt: 0.11610765755176544
epocht 0, batch_num 12600, step 12601, time: 1368.1995692253113 s, accu: 0.9674244523048401, loss_yt: 0.10331849753856659
epocht 0, batch_num 12800, step 12801, time: 1389.6610238552094 s, accu: 0.9674019813537598, loss_yt: 0.08799419552087784
epocht 0, batch_num 13000, step 13001, time: 1411.1642615795135 s, accu: 0.967383861541748, loss_yt: 0.0795389786362648
epocht 0, batch_num 13200, step 13201, time: 1432.5853960514069 s, accu: 0.9674171805381775, loss_yt: 0.09140339493751526
epocht 0, batch_num 13400, step 13401, time: 1453.9701228141785 s, accu: 0.9674357771873474, loss_yt: 0.08794796466827393
epocht 0, batch_num 13600, step 13601, time: 1475.4307816028595 s, accu: 0.9674654603004456, loss_yt: 0.052393339574337006
epocht 0, batch_num 13800, step 13801, time: 1496.859807729721 s, accu: 0.9674630165100098, loss_yt: 0.10222358256578445
epocht 0, batch_num 14000,

epocht 0, batch_num 10800, step 25603, time: 1168.4999060630798 s, accu: 0.9681666493415833, loss_yt: 0.079463891685009
epocht 0, batch_num 11000, step 25803, time: 1189.795717716217 s, accu: 0.9681711196899414, loss_yt: 0.06081700325012207
epocht 0, batch_num 11200, step 26003, time: 1211.317438840866 s, accu: 0.96817946434021, loss_yt: 0.062076278030872345
epocht 0, batch_num 11400, step 26203, time: 1232.604011774063 s, accu: 0.9681885838508606, loss_yt: 0.10052545368671417
epocht 0, batch_num 11600, step 26403, time: 1254.091302394867 s, accu: 0.9681947231292725, loss_yt: 0.07299789786338806
epocht 0, batch_num 11800, step 26603, time: 1275.5028581619263 s, accu: 0.9681949019432068, loss_yt: 0.060634732246398926
epocht 0, batch_num 12000, step 26803, time: 1296.9552941322327 s, accu: 0.9682009816169739, loss_yt: 0.06349457055330276
epocht 0, batch_num 12200, step 27003, time: 1318.2527792453766 s, accu: 0.9682068824768066, loss_yt: 0.10226769000291824
epocht 0, batch_num 12400, ste

epocht 0, batch_num 9200, step 38805, time: 993.9209911823273 s, accu: 0.9686657786369324, loss_yt: 0.06365431100130081
epocht 0, batch_num 9400, step 39005, time: 1015.3027074337006 s, accu: 0.9686669707298279, loss_yt: 0.09164287894964218
epocht 0, batch_num 9600, step 39205, time: 1036.7178659439087 s, accu: 0.9686759114265442, loss_yt: 0.06989862024784088
epocht 0, batch_num 9800, step 39405, time: 1058.0102016925812 s, accu: 0.968682587146759, loss_yt: 0.0859505906701088
epocht 0, batch_num 10000, step 39605, time: 1079.5768620967865 s, accu: 0.9686869382858276, loss_yt: 0.09296546876430511
epocht 0, batch_num 10200, step 39805, time: 1101.084318637848 s, accu: 0.9686927795410156, loss_yt: 0.07559355348348618
epocht 0, batch_num 10400, step 40005, time: 1122.448638677597 s, accu: 0.9687001705169678, loss_yt: 0.07925371080636978
epocht 0, batch_num 10600, step 40205, time: 1143.7558100223541 s, accu: 0.9687078595161438, loss_yt: 0.06876722723245621
epocht 0, batch_num 10800, step 4

epocht 0, batch_num 7600, step 52007, time: 821.4154300689697 s, accu: 0.969011127948761, loss_yt: 0.07514090090990067
epocht 0, batch_num 7800, step 52207, time: 842.8233268260956 s, accu: 0.9690178036689758, loss_yt: 0.058846306055784225
epocht 0, batch_num 8000, step 52407, time: 864.1154637336731 s, accu: 0.9690213799476624, loss_yt: 0.07439171522855759
epocht 0, batch_num 8200, step 52607, time: 885.3394162654877 s, accu: 0.9690219163894653, loss_yt: 0.064272940158844
epocht 0, batch_num 8400, step 52807, time: 906.7970468997955 s, accu: 0.9690254926681519, loss_yt: 0.0929197445511818
epocht 0, batch_num 8600, step 53007, time: 928.2297506332397 s, accu: 0.9690307378768921, loss_yt: 0.05221998691558838
epocht 0, batch_num 8800, step 53207, time: 949.6616535186768 s, accu: 0.9690297842025757, loss_yt: 0.06548319011926651
epocht 0, batch_num 9000, step 53407, time: 970.8731274604797 s, accu: 0.9690313339233398, loss_yt: 0.09423138201236725
epocht 0, batch_num 9200, step 53607, time:

...

epocht 2, batch_num 3600, step 210829, time: 393.82785987854004 s, accu: 0.9705373644828796, loss_yt: 0.07367337495088577
epocht 2, batch_num 3800, step 211029, time: 415.0008804798126 s, accu: 0.9705382585525513, loss_yt: 0.07879021018743515
epocht 2, batch_num 4000, step 211229, time: 436.3664240837097 s, accu: 0.9705383777618408, loss_yt: 0.05903554707765579
epocht 2, batch_num 4200, step 211429, time: 457.4941062927246 s, accu: 0.9705395102500916, loss_yt: 0.10157760232686996
epocht 2, batch_num 4400, step 211629, time: 478.6628062725067 s, accu: 0.9705394506454468, loss_yt: 0.05659033730626106
epocht 2, batch_num 4600, step 211829, time: 499.89226245880127 s, accu: 0.9705392122268677, loss_yt: 0.08456055074930191
epocht 2, batch_num 4800, step 212029, time: 521.306973695755 s, accu: 0.9705396294593811, loss_yt: 0.09023748338222504
epocht 2, batch_num 5000, step 212229, time: 542.6317825317383 s, accu: 0.9705398678779602, loss_yt: 0.051388029009103775
epocht 2, batch_num 5200, step

epocht 3, batch_num 2000, step 224031, time: 223.8474268913269 s, accu: 0.9705846309661865, loss_yt: 0.05761588737368584
epocht 3, batch_num 2200, step 224231, time: 245.0422830581665 s, accu: 0.9705851674079895, loss_yt: 0.05864004045724869
epocht 3, batch_num 2400, step 224431, time: 266.2434401512146 s, accu: 0.9705854654312134, loss_yt: 0.08983203023672104
epocht 3, batch_num 2600, step 224631, time: 287.5436305999756 s, accu: 0.9705860018730164, loss_yt: 0.0359339565038681
epocht 3, batch_num 2800, step 224831, time: 308.7733063697815 s, accu: 0.9705865979194641, loss_yt: 0.0727292150259018
epocht 3, batch_num 3000, step 225031, time: 330.1507761478424 s, accu: 0.9705871939659119, loss_yt: 0.067542165517807
epocht 3, batch_num 3200, step 225231, time: 351.47486901283264 s, accu: 0.9705875515937805, loss_yt: 0.06496710330247879
epocht 3, batch_num 3400, step 225431, time: 372.55325746536255 s, accu: 0.9705880880355835, loss_yt: 0.06763254106044769
epocht 3, batch_num 3600, step 225

epocht 3, batch_num 400, step 237233, time: 53.63523554801941 s, accu: 0.9706276655197144, loss_yt: 0.08807779848575592
epocht 3, batch_num 600, step 237433, time: 74.88936495780945 s, accu: 0.9706286191940308, loss_yt: 0.05373561754822731
epocht 3, batch_num 800, step 237633, time: 96.11623191833496 s, accu: 0.970629096031189, loss_yt: 0.08517591655254364
epocht 3, batch_num 1000, step 237833, time: 117.44735670089722 s, accu: 0.9706288576126099, loss_yt: 0.07440147548913956
epocht 3, batch_num 1200, step 238033, time: 138.48130917549133 s, accu: 0.9706291556358337, loss_yt: 0.06266561895608902
epocht 3, batch_num 1400, step 238233, time: 159.90034866333008 s, accu: 0.9706293940544128, loss_yt: 0.05148157849907875
epocht 3, batch_num 1600, step 238433, time: 181.15789985656738 s, accu: 0.9706297516822815, loss_yt: 0.04402722790837288
epocht 3, batch_num 1800, step 238633, time: 202.24710702896118 s, accu: 0.9706307053565979, loss_yt: 0.08894020318984985
epocht 3, batch_num 2000, step 

epocht 3, batch_num 14000, step 250833, time: 1498.2168254852295 s, accu: 0.9706575274467468, loss_yt: 0.08927468955516815
epocht 3, batch_num 14200, step 251033, time: 1519.4311270713806 s, accu: 0.970658540725708, loss_yt: 0.06364771723747253
epocht 3, batch_num 14400, step 251233, time: 1540.7958235740662 s, accu: 0.9706593155860901, loss_yt: 0.07846654206514359
epocht 3, batch_num 14600, step 251433, time: 1561.9832270145416 s, accu: 0.9706593155860901, loss_yt: 0.04813617840409279
epocht 3, batch_num 14800, step 251633, time: 1583.270447731018 s, accu: 0.9706598520278931, loss_yt: 0.038823358714580536
iter_validnum 3701
epochv 3, step 251634, stop_n 0, time: 1624.1005392074585 s, accu_va: 0.9706646827258667, loss_yv: 0.07361778085238545
iter_trainnum 14802
epocht 3, batch_num 0, step 251635, time: 11.499691247940063 s, accu: 0.9706684350967407, loss_yt: 0.0537378266453743
epocht 3, batch_num 200, step 251835, time: 32.69783091545105 s, accu: 0.9706693887710571, loss_yt: 0.10103522

KeyboardInterrupt: 

In [10]:
y_pred = modelcrnn.predict(testpd[feature])
fy_submission = np.squeeze(y_pred)
fy_submission = (fy_submission - fy_submission.min()) / (fy_submission.max() - fy_submission.min())

loading model...
INFO:tensorflow:Restoring parameters from ../data/particles/model/modelevery_mul4_modeltailv2/v2-252035


In [11]:
#阈值大概在0.2-0.4之间 本题对召回率较敏感，可适当降低一下阈值
thre = 0.5
#生成提交文件
sub = pd.DataFrame()
sub['hit_id'] = testpd['hit_id']
sub['flag_pred'] = fy_submission
sub['event_id'] = testpd['event_id']
sub['flag_pred'] = sub['flag_pred'].apply(lambda x: 1 if x >= thre else 0)
sub.to_csv(os.path.join(pathf, "subsample.csv").format(sub['flag_pred'].mean()), index=False)